# Классификация: SI > 8

In [1]:
import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

# Загрузка данных
df = pd.read_csv('/Users/rem/МИФИ/курсовая/dataset.csv')


features = [col for col in df.columns if col not in ["IC50, mM", "CC50, mM", "SI"]]
# Подготовка целевой переменной
y = (df["SI"] > 8).astype(int)
X = df[features]

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Для моделей на основе градиентного бустинга (LightGBM) необходимо очистить имена признаков от специальных символов
def clean_column(name):
    return re.sub(r'[^\w]', '_', name)

X_train_clean = X_train.rename(columns=lambda col: clean_column(col))
X_test_clean = X_test.rename(columns=lambda col: clean_column(col))


# Обучение моделей
logit = LogisticRegression(max_iter=1000)
logit.fit(X_train_scaled, y_train)
y_pred_logit = logit.predict(X_test_scaled)

rf = RandomForestClassifier()
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)

xgb = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
xgb.fit(X_train, y_train)
y_pred_xgb = xgb.predict(X_test)

lgb = LGBMClassifier()
lgb.fit(X_train_clean, y_train)
y_pred_lgb = lgb.predict(X_test_clean)

for name, preds, clf, xtest in [("Logistic Regression", y_pred_logit, logit, X_test_scaled),
                                ("Random Forest", y_pred_rf, rf, X_test),
                                ("XGBoost", y_pred_xgb, xgb, X_test),
                                ("LightGBM", y_pred_lgb, lgb, X_test)]:
    print(f"{name}: Accuracy={accuracy_score(y_test, preds):.3f}, Precision={precision_score(y_test, preds):.3f}, Recall={recall_score(y_test, preds):.3f}, F1={f1_score(y_test, preds):.3f}, AUC={roc_auc_score(y_test, clf.predict_proba(xtest)[:, 1]):.3f}")


/opt/anaconda3/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [01:33:35] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


[LightGBM] [Info] Number of positive: 273, number of negative: 499
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.004619 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 18123
[LightGBM] [Info] Number of data points in the train set: 772, number of used features: 170
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.353627 -> initscore=-0.603134
[LightGBM] [Info] Start training from score -0.603134
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, b

1. Logistic Regression

Accuracy = 0.711:
Модель правильно классифицирует 71,1% объектов, что является наивысшим показателем среди представленных моделей.

Precision = 0.600:
Из всех объектов, предсказанных как положительные, 60,0% действительно положительные. Это говорит о некотором количестве ложноположительных случаев.

Recall = 0.565:
Модель обнаруживает 56,5% всех реальных положительных случаев, что означает неплохую, но не выдающуюся способность захватывать все нужные объекты.

F1 = 0.582:
Баланс между precision и recall держится на среднем уровне — примерно 58,2%.

AUC = 0.703:
Это значение свидетельствует об умеренной способности логистической регрессии различать классы при изменении порога.

Интерпретация:
Логистическая регрессия показывает наивысшую accuracy, что может свидетельствовать о том, что её решения при фиксированном пороге дают достаточно верные результаты. Однако относительная невысокая полнота (Recall) и AUC указывают, что при изменении порога модель не так хорошо различает положительные и отрицательные случаи.

2. Random Forest

Accuracy = 0.691:
Общая точность чуть ниже — 69,1%, что немного уступает Logistic Regression.

Precision = 0.579:
Из предсказанных положительных объектов 57,9% являются истинно положительными, что немного хуже логистической регрессии.

Recall = 0.478:
Модель обнаруживает лишь 47,8% реальных положительных случаев — значение заметно ниже, что указывает на проблематику в находке всех позитивов.

F1 = 0.524:
Гармоническое среднее между precision и recall также ниже, чем у Logistic Regression, отражая общую потерю в балансе между точностью и полнотой.

AUC = 0.731:
Несмотря на несколько худшие показатели при фиксированном пороге, AUC выше (0.731). Это говорит о лучшей способности модели ранжировать объекты, даже если выбранный порог не оптимален.

Интерпретация:
Random Forest работает несколько осторожнее, что выражается в меньшей полноте (то есть модель с большей вероятностью пропускает реальные положительные примеры), но при этом способность различать классы (AUC) улучшена по сравнению с Logistic Regression. Это может указывать на потенциал модели при дополнительной настройке порогов или весов.

3. XGBoost

Accuracy = 0.696:
Общая точность составляет 69,6%, что немного ниже показателя Logistic Regression, но сопоставимо с Random Forest.

Precision = 0.566:
Из предсказанных положительных только 56,6% являются истинными положительными, то есть модель склонна выдавать больше ложноположительных результатов.

Recall = 0.623:
При этом полнота равна 62,3% — самое высокое значение среди моделей, что свидетельствует о лучшем обнаружении всех позитивных случаев.

F1 = 0.593:
Баланс между precision и recall чуть улучшен (59,3%), что говорит о том, что модель работает с небольшим перекосом в сторону повышения обнаружения позитивов за счёт точности.

AUC = 0.735:
Это один из лучших показателей AUC, указывающий на высокую способность модели различать классы по вероятностной шкале.

Интерпретация:
XGBoost демонстрирует лучший recall, то есть успешно находит больше истинных положительных примеров, что может быть особенно актуально в задачах, где критично не пропустить позитивы. Несмотря на несколько более низкую precision, AUC остаётся высоким, что говорит о хорошем ранжировании объектов по принадлежности к классам. Таким образом, модель эффективна для задач, где важна полнота, даже если это идет в ущерб точности предсказаний.

4. LightGBM

Accuracy = 0.675:
Общая точность равна 67,5%, что является самым низким значением среди рассмотренных моделей.

Precision = 0.548:
Из предсказанных положительных 54,8% действительно положительные – значение precision ниже, чем у остальных моделей.

Recall = 0.493:
Полнота составляет 49,3%, что сигнализирует о том, что модель пропускает более половины реальных положительных примеров.

F1 = 0.519:
Сбалансированный показатель между precision и recall также самый низкий — всего 51,9%.

AUC = 0.736:
Интересно, что AUC LightGBM оказывается самым высоким (или сопоставимым с лучшими) — 0.736, что говорит о том, что при варьировании порога модель способна хорошо разделять классы. Однако при фиксированном пороге ее предсказания оказываются менее точными.

Интерпретация:
Несмотря на лучший показатель AUC, оценка по фиксированному порогу (accuracy, precision, recall, F1) для LightGBM оказалась самой низкой. Это может означать, что оптимальный порог для принятия решения не был подобран, либо модель требует дополнительной калибровки. Высокий AUC указывает, что LightGBM имеет потенциал для различения классов, если правильно настроить пороговое значение.

Итоговое сравнение и рекомендации

Фиксированный порог vs. способность ранжирования:

Logistic Regression показывает наивысшую accuracy (0.711) и хорошие сбалансированные показатели (precision = 0.600, recall = 0.565, F1 = 0.582), но имеет самый низкий AUC (0.703).
Random Forest уступает Logistic Regression по accuracy, precision и recall, но выигрывает по AUC (0.731), что свидетельствует о потенциале модели при оптимальном выборе порога.
XGBoost демонстрирует самый высокий recall (0.623) и один из лучших показателей AUC (0.735). Это может быть полезно, если задача требует обнаружения всех позитивных случаев, даже если точность предсказаний немного ниже.
LightGBM показывает худшие значения по пороговым метрикам (accuracy, precision, recall, F1), но имеет самый высокий AUC (0.736). Это указывает на то, что LightGBM хорошо различает классы в ранжировании, однако оптимизация порога или калибровка вероятностей может повысить её практическую эффективность.
Выбор модели зависит от задачи:
Если требуется максимальная точность фиксированного порога, то Logistic Regression оказывается наиболее стабильной.
Если важнее не пропустить позитивы, XGBoost с его высоким recall является предпочтительным выбором, несмотря на снижение precision.
Если планируется дальнейшая настройка порога для повышения разделительной способности, модели с высоким AUC (Random Forest и LightGBM) могут дать преимущества после дополнительной оптимизации.

Таким образом, каждая модель имеет свои сильные и слабые стороны. Итоговый выбор модели следует делать исходя из специфики задачи и требований: например, снижать вероятность пропуска важных случаев (Recall) или минимизировать ложноположительные предсказания (Precision), а также учитывать возможность дальнейшей настройки порогов, опираясь на значение AUC.

